<a href="https://colab.research.google.com/github/sgokulakannankg19/ClinicalTrialFiles/blob/main/API_run2_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade gspread

  Attempting uninstall: gspread
    Found existing installation: gspread 3.4.2
    Uninstalling gspread-3.4.2:
      Successfully uninstalled gspread-3.4.2


In [ ]:
from google.colab import auth
auth.authenticate_user()

import pandas as pd
import requests
from collections import Counter


import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
BASE_URL = "https://clinicaltrials.gov/api/query/study_fields?expr="
expression = "cancer"
format = "json"
fields = "NCTId"
final_url = BASE_URL + f"{expression}&fields={fields}&min_rnk=1&max_rnk=&fmt={format}" 

In [ ]:
final_url

'https://clinicaltrials.gov/api/query/study_fields?expr=cancer&fields=NCTId&min_rnk=1&max_rnk=&fmt=json'

In [ ]:
response = requests.get(final_url)
response.content

b'{\n  "StudyFieldsResponse":{\n    "APIVrs":"1.01.05",\n    "DataVrs":"2022:02:02 22:53:24.015",\n    "Expression":"cancer",\n    "NStudiesAvail":403296,\n    "NStudiesFound":98989,\n    "MinRank":1,\n    "MaxRank":20,\n    "NStudiesReturned":20,\n    "FieldList":[\n      "NCTId"\n    ],\n    "StudyFields":[\n      {\n        "Rank":1,\n        "NCTId":[\n          "NCT04318756"\n        ]\n      },{\n        "Rank":2,\n        "NCTId":[\n          "NCT03581708"\n        ]\n      },{\n        "Rank":3,\n        "NCTId":[\n          "NCT04834570"\n        ]\n      },{\n        "Rank":4,\n        "NCTId":[\n          "NCT00897650"\n        ]\n      },{\n        "Rank":5,\n        "NCTId":[\n          "NCT04494620"\n        ]\n      },{\n        "Rank":6,\n        "NCTId":[\n          "NCT02890667"\n        ]\n      },{\n        "Rank":7,\n        "NCTId":[\n          "NCT04875351"\n        ]\n      },{\n        "Rank":8,\n        "NCTId":[\n          "NCT00512551"\n        ]\n      },{\

In [ ]:
df_result = pd.read_json(response.content)
df_result

,StudyFieldsResponse
APIVrs,1.01.05
DataVrs,2022:02:02 22:53:24.015
Expression,cancer
FieldList,[NCTId]
MaxRank,20
MinRank,1
NStudiesAvail,403296
NStudiesFound,98989
NStudiesReturned,20
StudyFields,"[{'Rank': 1, 'NCTId': ['NCT04318756']}, {'Rank..."


In [ ]:
end_limit = df_result.loc['NStudiesFound', 'StudyFieldsResponse']
end_limit

98989

In [ ]:
ss = gc.create("CT_test","1rSPoezkw1yRVb9YKeJPZe9IuOF9YaMTT")

In [ ]:
fields = "NCTId,StudyType,OverallStatus,Phase,OfficialTitle,Condition,InterventionName,PrimaryOutcomeMeasure,PrimaryOutcomeDescription,SecondaryOutcomeMeasure,SecondaryOutcomeDescription,OtherOutcomeMeasure,OtherOutcomeDescription,ReferencePMID,ConditionMeshTerm,ConditionAncestorTerm,InterventionMeshTerm,InterventionAncestorTerm"
# fields = "NCTId,PrimaryOutcomeDescription,SecondaryOutcomeDescription,OtherOutcomeDescription"
# fields = "InterventionType,ArmGroupInterventionName"
column_headings = ["NCT ID", "Study Type", "Recruitment Status", "Phase", "Official Title", "Cancer Type", "Intervention", "Primary OM Title", "Primary OM Description", "Secondary OM Title", "Secondary OM Description", "Other OM Title", "Other OM Description", "PMID", "MeSH Terms", "Character Count", "URL"]

# column_headings = ["Interventions","Drug"]

def create_spreadsheet(min_value, max_value):
  nct_list = []
  result_list = []
  final_url = BASE_URL + f"{expression}&fields={fields}&min_rnk={min_value}&max_rnk={max_value}&fmt={format}" 
  print(final_url)
  response = requests.get(final_url)
  print(response)
  df_json = pd.read_json(response.content)
  df_json['StudyFieldsResponse']['StudyFields']
  for data in df_json['StudyFieldsResponse']['StudyFields']:
    # print(data)
    trial_rows=[]

    nct_id = ','.join(data["NCTId"])
    study_type = ','.join(data["StudyType"])
    overall_status = ','.join(data["OverallStatus"])
    phase = '/'.join(data["Phase"])
    official_title = ','.join(data["OfficialTitle"])
    condition = '\n'.join(data["Condition"])
    intervention = ' || '.join(data["InterventionName"])
    primary_outcome_measure = '\n'.join(data["PrimaryOutcomeMeasure"])
    primary_outcome_description = '\n\n'.join(data["PrimaryOutcomeDescription"])
    secondary_outcome_measure = "\n".join(data["SecondaryOutcomeMeasure"])
    secondary_outcome_description = "\n\n".join(data["SecondaryOutcomeDescription"])
    other_outcome_measure = "\n".join(data["OtherOutcomeMeasure"])
    other_outcome_description = "\n\n".join(data["OtherOutcomeDescription"])
    pmid = str(' || '.join(data["ReferencePMID"]))
    condition_mesh_term = data["ConditionMeshTerm"]
    condition_ancestor_term = data["ConditionAncestorTerm"]
    intervention_mesh_term = data["InterventionMeshTerm"]
    intervention_ancestor_term = data["InterventionAncestorTerm"]

    if (len(primary_outcome_description)) >= 50000:
      character_count = "CHECK OM Description"
    elif (len(secondary_outcome_description)) >= 50000:
      character_count = "CHECK OM Description"
    elif (len(other_outcome_description)) >= 50000:
      character_count = "CHECK OM Description"
    else:
      character_count = ""
  
    
    mesh_terms_list = condition_mesh_term + condition_ancestor_term + intervention_mesh_term + intervention_ancestor_term
    mesh_terms = ",".join(mesh_terms_list)

    trial_rows.append(nct_id)
    trial_rows.append(study_type)
    trial_rows.append(overall_status)
    trial_rows.append(phase)
    trial_rows.append(official_title)
    trial_rows.append(condition)
    trial_rows.append(intervention)
    trial_rows.append(primary_outcome_measure)
    trial_rows.append(primary_outcome_description[:50000])
    trial_rows.append(secondary_outcome_measure)
    trial_rows.append(secondary_outcome_description[:50000])
    trial_rows.append(other_outcome_measure)
    trial_rows.append(other_outcome_description[:50000])
    trial_rows.append(pmid)
    trial_rows.append(mesh_terms)
    trial_rows.append(character_count)
    trial_rows.append("https://clinicaltrials.gov/ct2/show/" + ''.join(nct_id))
    
    result_list.append(trial_rows)

  result_list_df = pd.DataFrame(result_list)
  result_list_df.columns = column_headings

  list_of_lists = result_list_df.to_numpy().tolist()
  headers = result_list_df.columns.tolist()

  data_to_write = [headers] + list_of_lists

  sh = gc.open("CT_Run2_03022022")
  worksheet = sh.add_worksheet(title=f"Rank{min_value} - {max_value}", rows="1002", cols="30")
  worksheet.update("A1",data_to_write,value_input_option="USER_ENTERED")
    

In [ ]:
min_value = 1
max_value = 1000

create_spreadsheet(97001, 98000)

# # list_of_id = []
# while max_value <= end_limit:

#   create_spreadsheet(min_value, max_value)
#   print("done")
  
#   min_value += 1000
#   max_value += 1000
# if max_value != end_limit:
#   min_value = min_value
#   max_value = end_limit

#   create_spreadsheet(min_value, max_value)
# #   list_of_id.append(data)

# # print(list_of_id)
# # print(len(list_of_id))

https://clinicaltrials.gov/api/query/study_fields?expr=cancer&fields=NCTId,StudyType,ExpAccTypeIndividual,ExpAccTypeIntermediate,ExpAccTypeTreatment&min_rnk=97001&max_rnk=98000&fmt=json
<Response [200]>


























































































































































































































Yes































































































































































































































































































































































































































































































































































































